Mein Beispiel wird hier stattfinden. Mit der neuen tpch database und einer einfachen query zuerst.

In [1]:
import pathlib
from inspect import cleandoc
from mlinspect import PipelineInspector
from mlinspect.utils import get_project_root
from mlinspect.to_sql.dbms_connectors.postgresql_connector import PostgresqlConnector
import os
import time

##Mal schauen ob sich das problem löst, wenn ich alle sachen importiere
from mlinspect.inspections import HistogramForColumns, RowLineage, MaterializeFirstOutputRows
from mlinspect.checks import NoBiasIntroducedFor, NoIllegalFeatures
from demo.feature_overview.no_missing_embeddings import NoMissingEmbeddings
from example_pipelines.healthcare import custom_monkeypatching
from mlinspect import OperatorType
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    

    


POSTGRES_USER = "postgres"
POSTGRES_PW = "123456"
POSTGRES_DB = "db1"
POSTGRES_PORT = 25432
POSTGRES_HOST = "localhost"

#POSTGRES_USER = "luca"
#POSTGRES_PW = "password"
#POSTGRES_DB = "healthcare_benchmark"
#POSTGRES_PORT = 5432
#POSTGRES_HOST = "localhost"

dbms_connector_p = PostgresqlConnector(dbname=POSTGRES_DB, user=POSTGRES_USER, password=POSTGRES_PW,
                                       port=POSTGRES_PORT, host=POSTGRES_HOST)

print(dbms_connector_p.db_settings)




{'dbname': 'db1', 'user': 'postgres', 'password': '123456', 'port': 25432, 'host': 'localhost'}


In [2]:
pipeline_code = cleandoc("""
    import warnings
    import os
    import pandas as pd
    import numpy as np
    from sklearn.compose import ColumnTransformer
    from sklearn.impute import SimpleImputer
    from sklearn.model_selection import train_test_split
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import OneHotEncoder, StandardScaler
    from example_pipelines.healthcare.healthcare_utils import MyW2VTransformer, MyKerasClassifier, \
        create_model
    from mlinspect.utils import get_project_root, store_timestamp
    import time
    from mlinspect.to_sql.dbms_connectors.postgresql_connector import PostgresqlConnector
    
    #HIER IST DIE ECHTE TESTDB MIT TPCH DATEN
    POSTGRES_USER = "postgres"
    POSTGRES_PW = "123456"
    POSTGRES_DB = "db1"
    POSTGRES_PORT = 25432
    POSTGRES_HOST = "localhost"
    
    dbms_connector_p = PostgresqlConnector(dbname=POSTGRES_DB, user=POSTGRES_USER, password=POSTGRES_PW, port=POSTGRES_PORT, host=POSTGRES_HOST)
    
    def udf_disc_price(extended, discount):
        return (extended*(1-discount))

    def udf_charge(extended, discount, tax):
        return extended*((1-discount)*(1+tax))
        #np.multiply(extended, np.multiply(np.subtract(1, discount), np.add(1, tax)))
               
    def load_table(table, sf=1):
        parse_dates=[]
        if table=='orders':
            parse_dates=['o_orderdate']
        elif table=='lineitem':
            parse_dates=['l_shipdate', 'l_commitdate', 'l_receiptdate']

        df = pd.read_sql_query('SELECT * FROM pg2_sf'+str(sf)+'_'+table,
                             dbms_connector_p.connection, parse_dates=parse_dates)
        dbms_connector_p.run(f"END;")
        
        #cols = df.select_dtypes(object).columns
        #df[cols] = df[cols].apply(lambda x: x.astype(str).str.strip())
        return df
   
    #XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
    #Query 1
    #parse_dates=['l_shipdate', 'l_commitdate', 'l_receiptdate']
    #lineitem=pd.read_sql_query('SELECT * FROM pg2_sf1_lineitem', dbms_connector_p.connection, parse_dates=parse_dates)
    #
    #df=lineitem[["l_shipdate", "l_returnflag", "l_linestatus", "l_quantity",
    #                "l_extendedprice", "l_discount", "l_tax"]][(lineitem['l_shipdate'] <= '1998-09-01')]
    #df['disc_price'] = udf_disc_price(
    #    df['l_extendedprice'], df['l_discount'])
    #df['charge'] = udf_charge(df['l_extendedprice'],
    #                            df['l_discount'], df['l_tax'])    
    #res = df.groupby(['l_returnflag', 'l_linestatus'])\
    #    .agg({'l_quantity': 'sum', 'l_extendedprice': 'sum', 'disc_price': 'sum', 'charge': 'sum',
    #            'l_quantity': 'mean', 'l_extendedprice': 'mean', 'l_discount': 'mean', 'l_shipdate': 'count'})
    #
    #print("df jjjjjjjjjjjjjjjjjjjjjjjjjjjj", df)
    #print("res jjjjjjjjjjjjjjjjjjjjjjjjjj", res)
    
    \"\"\"
    #Query 1 with methods
    lineitem = load_table('lineitem', 1)
    
    df = lineitem[["l_shipdate", "l_returnflag", "l_linestatus", "l_quantity",
                   "l_extendedprice", "l_discount", "l_tax"]][(lineitem['l_shipdate'] <= '1998-09-01')]
    df['disc_price'] = udf_disc_price(
        df['l_extendedprice'], df['l_discount'])
    df['charge'] = udf_charge(df['l_extendedprice'],
                              df['l_discount'], df['l_tax'])
    res = df.groupby(['l_returnflag', 'l_linestatus'])\
        .agg({'l_quantity': 'sum', 'l_extendedprice': 'sum', 'disc_price': 'sum', 'charge': 'sum',
              'l_quantity': 'mean', 'l_extendedprice': 'mean', 'l_discount': 'mean', 'l_shipdate': 'count'})
    
    print("df jjjjjjjjjjjjjjjjjjjjjjjjjjjj", df)
    print("res jjjjjjjjjjjjjjjjjjjjjjjjjj", res)
    
    
    
    
    \"\"\"
    
    #Query 2
    nation = load_table('nation', 1)
    part = load_table('part', 1)
    supplier = load_table('supplier', 1)
    region = load_table('region', 1)
    partsupp = load_table('partsupp', 1)
    
    ps = partsupp[["ps_partkey", "ps_suppkey", "ps_supplycost"]]
    p = part[["p_partkey", "p_mfgr", "p_size", "p_type"]][(part['p_size'] == 15) & (
        part['p_type'].str.match(".*BRASS$"))][["p_partkey", "p_mfgr"]]
        
    psp = ps.merge(p, left_on="ps_partkey", right_on="p_partkey")
    s = supplier[["s_suppkey", "s_nationkey", "s_acctbal",
                  "s_name", "s_address", "s_phone", "s_comment"]]
    psps = psp.merge(s, left_on="ps_suppkey", right_on="s_suppkey")[
        ["ps_partkey", "ps_supplycost", "p_mfgr", "s_nationkey",         "s_acctbal", "s_name", "s_address", "s_phone", "s_comment"]]
    nr = nation.merge(region[region["r_name"] == "EUROPE"], left_on="n_regionkey", right_on="r_regionkey")[["n_nationkey", "n_name"]]
    
    pspsnr = psps.merge(nr, left_on="s_nationkey", right_on="n_nationkey")[["ps_partkey", "ps_supplycost", "p_mfgr", "n_name", "s_acctbal", "s_name", "s_address", "s_phone", "s_comment"]]
    aggr = pspsnr.groupby("ps_partkey").agg({'ps_supplycost': min})#.reset_index()
    sj = pspsnr.merge(aggr, left_on=["ps_partkey", "ps_supplycost"], right_on=["ps_partkey", "ps_supplycost"])
    res = sj[["s_acctbal", "s_name", "n_name", "ps_partkey", "p_mfgr", "s_address", "s_phone", "s_comment"]].sort_values(["s_acctbal", "n_name", "s_name", "ps_partkey"], ascending=[False, True, True, True]).head(100)
    
    #res = sj[["s_acctbal", "s_name", "n_name", "ps_partkey", "p_mfgr", "s_address", "s_phone", "s_comment"]].head(100)
    
    
    
    print("res jjjjjjjjjjjjjjjjjjjjjjjjjj", res)
    

    
""")

print(pipeline_code)

import warnings
import os
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from example_pipelines.healthcare.healthcare_utils import MyW2VTransformer, MyKerasClassifier,         create_model
from mlinspect.utils import get_project_root, store_timestamp
import time
from mlinspect.to_sql.dbms_connectors.postgresql_connector import PostgresqlConnector

#HIER IST DIE ECHTE TESTDB MIT TPCH DATEN
POSTGRES_USER = "postgres"
POSTGRES_PW = "123456"
POSTGRES_DB = "db1"
POSTGRES_PORT = 25432
POSTGRES_HOST = "localhost"

dbms_connector_p = PostgresqlConnector(dbname=POSTGRES_DB, user=POSTGRES_USER, password=POSTGRES_PW, port=POSTGRES_PORT, host=POSTGRES_HOST)

def udf_disc_price(extended, discount):
    return (extended*(1-discount))

def udf_charge(extended, discount, 

In [3]:
## The function to retrieve the generated code:
def get_sql_query(pipeline_code, to_sql, dbms_connector=None, mode=None, materialize=None):
    from PIL import Image
    import matplotlib.pyplot as plt
    from mlinspect.visualisation import save_fig_to_path
    import os
    import sys
    
    inspector_result = PipelineInspector \
        .on_pipeline_from_string(pipeline_code) 
    
    if to_sql:
        inspector_result = inspector_result.execute_in_sql(dbms_connector=dbms_connector, mode=mode, materialize=materialize)
    else:
        inspector_result = inspector_result.execute()
        return None

    #extracted_dag = inspector_result.dag
    #filename = os.path.join(str(get_project_root()), "demo", "feature_overview", "newTest.png")
    #save_fig_to_path(extracted_dag, filename)
    #im = Image.open(filename)
    #plt.imshow(im)
    
    #test_file = \
    #    pathlib.Path(get_project_root() / r"mlinspect/to_sql/generated_code/pipeline.sql")
    
    #with test_file.open("r") as file:
    #    test__code = file.read()

    #return test__code
    return None

In [4]:
#dbms_connector_p = PostgresqlConnector(dbname=POSTGRES_DB, user=POSTGRES_USER, password=POSTGRES_PW,
#                                       port=POSTGRES_PORT, host=POSTGRES_HOST)
#print(dbms_connector_p.db_settings)

def run_for_all(pipeline_code, mode="", materialize=None):
    
#    dbms_connector_p = PostgresqlConnector(dbname=POSTGRES_DB, user=POSTGRES_USER, password=POSTGRES_PW,
#                                       port=POSTGRES_PORT, host=POSTGRES_HOST)
    print(dbms_connector_p.db_settings)

    t0 = time.time()
    test_code = get_sql_query(pipeline_code=pipeline_code, to_sql=True, dbms_connector=dbms_connector_p, mode=mode, materialize=materialize)
    t1 = time.time()
    print("\nTime spend with modified SQL inspections (PSQL): " + str(t1 - t0))
    
    #t2 = time.time()
    #result= get_sql_query(pipeline_code=pipeline_code, to_sql=False)
    #t3 = time.time()
    #print("\nTime spend with original (pandas): " + str(t3 - t2))

    #(also eventuell könnte ich hier statt get_sql_query oben diesen befehl get_sql_query_for_pipeline von code_as_string.py in example_to_sql benutzen, damit der setup_code und test_code in die files geschrieben wird, aber noch nicht ausgeführt wird!!!)
    #(und dann diesen befehl hier laufen lassen, damit die tabellen nicht neu im dbms erzeugt werden) dbms_connector_p.run(test_code)
    #print(setup_code + "\n" + test_code)
    
    

In [5]:
# The code generated using "CTE":

#run_for_all(pipeline_code, mode="CTE", materialize=False)
    

In [6]:
# The code generated using "VIEW":

run_for_all(pipeline_code, mode="VIEW", materialize=False)



{'dbname': 'db1', 'user': 'postgres', 'password': '123456', 'port': 25432, 'host': 'localhost'}
TTTTTTTTTTTTTTTTTTTTT hier ist der modified ast
Module(body=[ImportFrom(module='mlinspect.instrumentation._pipeline_executor', names=[alias(name='set_code_reference_call', asname=None), alias(name='set_code_reference_subscript', asname=None), alias(name='monkey_patch', asname=None), alias(name='undo_monkey_patch', asname=None)], level=0), Expr(value=Call(func=Name(id='monkey_patch', ctx=Load()), args=[], keywords=[])), Import(names=[alias(name='warnings', asname=None)]), Import(names=[alias(name='os', asname=None)]), Import(names=[alias(name='pandas', asname='pd')]), Import(names=[alias(name='numpy', asname='np')]), ImportFrom(module='sklearn.compose', names=[alias(name='ColumnTransformer', asname=None)], level=0), ImportFrom(module='sklearn.impute', names=[alias(name='SimpleImputer', asname=None)], level=0), ImportFrom(module='sklearn.model_selection', names=[alias(name='train_test_split', 

OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO ICH WILL WISSEN OB PATCHED_READ_sql_query ausgelöst wird :D
RRRRRRRRRRRRRRRRRRRRRRRRRR original <function read_sql_query at 0x0000019622F10C10>
TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT2
üüüüüüüüüüüüüüüüüüüüüüüüüüüüüüüüüüüü ergebnis_source_code     df = pd.read_sql_query('SELECT * FROM pg2_sf'+str(sf)+'_'+table,
                         dbms_connector_p.connection, parse_dates=parse_dates)
üüüüüüüüüüüüüüüüüüüüüüüüüüüüüüüüüüüü caller_source_code in monkeypatching,  pd.read_sql_query('SELECT * FROM pg2_sf'+str(sf)+'_'+table,
                         dbms_connector_p.connection, parse_dates=parse_dates)
üüüüüüüüüüüüüüüüüüüüüüüüüüüüüüüüüü alle argumente Original_func <function read_sql_query at 0x0000019622F10C10>
üüüüüüüüüüüüüüüüüüüüüüüüüüüüüüüüüü alle argumente execute_inspections_func <function PandasPatchingSQL.patched_read_sql_query.<locals>.execute_inspections at 0x00000196451323A0>
üüüüüüüüüüüüüüüüüüüüüüüüüüüüüüüüüü alle argument

üüüüüüüüüüüüüüüüüüüüüüüüüüüüüüüüüü alle argumente *args      p_partkey                                 p_name  \
0          782   peru firebrick coral chartreuse rosy   
1          783         cyan turquoise azure pink dark   
2          784   light smoke seashell snow chartreuse   
3          785       coral saddle indian lime frosted   
4          786      royal azure ivory moccasin salmon   
..         ...                                    ...   
995       1777    burnished black almond white violet   
996       1778     honeydew sky salmon misty lavender   
997       1779  rose wheat cream chartreuse goldenrod   
998       1780          linen metallic smoke sky puff   
999       1781          black indian forest hot olive   

                        p_mfgr     p_brand                  p_type  p_size  \
0    Manufacturer#1             Brand#13      LARGE POLISHED BRASS      29   
1    Manufacturer#1             Brand#13       SMALL PLATED NICKEL       4   
2    Manufacturer#1      

result  0      False
1      False
2      False
3      False
4      False
       ...  
995    False
996    False
997    False
998    False
999    False
Name: op_14, Length: 1000, dtype: bool
tracking_cols  ['pg2_sf1_part_ctid']
non_tracking_cols  None
origin_context  <mlinspect.to_sql.py_to_sql_mapping.OpTree object at 0x000001964586AEE0>
cte_name  
PPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPP  Hier sind wir in Wrap_in_sql_obj in sql_logic PPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPP
sqlcode SELECT ((("p_size") = (15)) AND (("p_type") LIKE ('%BRASS'))) AS op_14, pg2_sf1_part_ctid
FROM pg2_sf1_part_2_mlinid1_ctid
block_name 
Und hier ist das ergebnis der beiden werte
sqlcode  CREATE VIEW block_mlinid11_15 AS (
	SELECT ((("p_size") = (15)) AND (("p_type") LIKE ('%BRASS'))) AS op_14, pg2_sf1_part_ctid
	FROM pg2_sf1_part_2_mlinid1_ctid
);

block_name  block_mlinid11_15
Hier will ich kurz wissen ob die Tableinfo verändert wurde
altes result tableinfo 0      False
1      False
2      F

zzzzzzzzzzzzzzzzzzz hier ist das jeweilige df in self.mapping in echt ('block_mlinid11_15', TableInfo(data_object=0      False
1      False
2      False
3      False
4      False
       ...  
995    False
996    False
997    False
998    False
999    False
Name: op_14, Length: 1000, dtype: bool, tracking_cols=['pg2_sf1_part_ctid'], non_tracking_cols='"op_14"', operation_type=<OperatorType.BIN_OP: 'Arithmetic or Boolean Operation'>, origin_context=<mlinspect.to_sql.py_to_sql_mapping.OpTree object at 0x000001964586AEE0>))
zzzzzzzzzzzzzzzzzzz hier ist das jeweilige df in self.mapping in echt ('block_mlinid10_13', TableInfo(data_object=0       True
1      False
2      False
3      False
4      False
       ...  
995    False
996    False
997     True
998    False
999     True
Name: op_12, Length: 1000, dtype: bool, tracking_cols=['pg2_sf1_part_ctid'], non_tracking_cols='"op_12"', operation_type=<OperatorType.BIN_OP: 'Arithmetic or Boolean Operation'>, origin_context=<mlinspect.to_sql.py_to

zzzzzzzzzzzzzzzzzzz hier ist das jeweilige df in self.mapping in echt ('pg2_sf1_partsupp_5_mlinid4_ctid', TableInfo(data_object=     ps_partkey  ps_suppkey  ps_availqty  ps_supplycost  \
0           181        7682         1122         657.25   
1           182         183         9699         535.27   
2           182        2683          960         519.36   
3           182        5183         6243         741.46   
4           182        7683         6146         365.00   
..          ...         ...          ...            ...   
995         429        2930         6398         787.57   
996         429        5430         3641          43.51   
997         429        7930         9468         312.91   
998         430         431           15         483.73   
999         430        2931         7890         357.47   

                                            ps_comment  
0    lyly fluffily pending foxes. fluffily ironic p...  
1    ound the furiously regular foxes. pending re

zzzzzzzzzzzzzzzzzzz hier ist das jeweilige df in self.mapping in echt ('block_mlinid5_6', TableInfo(data_object=     ps_partkey  ps_suppkey  ps_supplycost
0           181        7682         657.25
1           182         183         535.27
2           182        2683         519.36
3           182        5183         741.46
4           182        7683         365.00
..          ...         ...            ...
995         429        2930         787.57
996         429        5430          43.51
997         429        7930         312.91
998         430         431         483.73
999         430        2931         357.47

[1000 rows x 3 columns], tracking_cols=['pg2_sf1_partsupp_ctid'], non_tracking_cols=['"ps_partkey"', '"ps_suppkey"', '"ps_supplycost"'], operation_type=<OperatorType.PROJECTION: 'Projection'>, origin_context=<mlinspect.to_sql.py_to_sql_mapping.OpTree object at 0x000001964583AB20>))
zzzzzzzzzzzzzzzzzzz hier ist das jeweilige df in self.mapping in echt ('pg2_sf1_partsupp

ENDLICH DAS SQL_RESULT     s_acctbal                     s_name                     n_name  \
0     9938.53  Supplier#000005359         UNITED KINGDOM              
1     9937.84  Supplier#000005969         ROMANIA                     
2     9936.22  Supplier#000005250         UNITED KINGDOM              
3     9923.77  Supplier#000002324         GERMANY                     
4     9871.22  Supplier#000006373         GERMANY                     
..        ...                        ...                        ...   
95    7887.08  Supplier#000009792         GERMANY                     
96    7871.50  Supplier#000007206         RUSSIA                      
97    7852.45  Supplier#000005864         RUSSIA                      
98    7850.66  Supplier#000001518         UNITED KINGDOM              
99    7843.52  Supplier#000006683         FRANCE                      

    ps_partkey                     p_mfgr  \
0       185358  Manufacturer#4              
1       108438  Manufacturer#1    

In [7]:
# The code generated using "VIEW" and "MATERIALIZED":

#run_for_all(pipeline_code, mode="VIEW", materialize=True)